In [21]:
from brainlit.utils import read_swc, df_to_graph, graph_to_paths
from brainlit.viz.visualize import napari_viewer
from mouselight_code.src import swc2voxel
import numpy as np
from skimage import io
from scipy.ndimage.morphology import distance_transform_edt
from pathlib import Path

In [22]:
brain_offsets = {
    "10-01": [69445.19581378, 12917.40798423, 30199.63896704],
    "8-01": [70093.27584462, 15071.5958194, 29306.73645404],
}
vol_offsets = {
    "10-01": {
        1: [3944.427317, 1689.489974, 2904.058044],
        2: [7562.41721, 2517.659516, 6720.099583],
        3: [6440.344565, 3724.653335, 3315.921558],
        4: [3693.850008, 4690.851133, 4759.545202],
        5: [2176.050385, 4472.356622, 5422.519379],
        6: [5186.880558, 1607.205131, 5627.930585],
        7: [4474.380558, 3801.205131, 5641.030585],
        8: [8625.680558, 3461.805131, 7853.730585],
        9: [8036.380558, 2739.005131, 7646.730585],
        10: [8908.480558, 2241.305131, 5275.430585],
        11: [5763.576767, 920.389294, 6949.146129],
        12: [4395.108079, 3142.101761, 7674.109968],
        13: [6357.017903, 3962.134266, 1793.497497],
        14: [1290.816602, 3784.927683, 5489.762402],
        15: [3261.686282, 4042.901892, 2753.811915],
        16: [6434.371327, 3146.622337, 5511.826519],
        17: [8759.453985, 3140.594903, 8062.858693],
        18: [3647.85608, 4787.29009, 5026.415948],
        19: [6278.469831, 1981.820562, 2234.779833],
        20: [2202.332629, 3856.654157, 2746.457209],
        21: [6119.880378, 3134.567468, 4973.882337],
        22: [7348.575311, 1596.96885, 3394.721975],
        23: [6633.877456, 2001.108354, 7079.429486],
        24: [6226.801328, 4177.61506, 3591.197682],
        25: [6270.405961, 4555.535222, 3526.056004],
    },
    "8-01": {
        1: [3808.881423, 2359.223225, 5006.26702],
        2: [6889.089085, 2566.530453, 8891.683733],
        3: [7128.395228, 1863.63414, 9648.801312],
        4: [5482.231871, 4208.245402, 11971.55106],
        5: [6671.293606, 3960.755275, 9643.859292],
        6: [5762.10132, 3843.673284, 5200.517052],
        7: [6337.40132, 4651.873284, 3297.317052],
        8: [5744.50132, 2051.973284, 5214.317052],
        9: [4518.90132, 1426.273284, 6342.517052],
        10: [4765.10132, 4331.973284, 9779.817052],
        11: [8681.046946, 3866.08193, 9473.853778],
        12: [3311.148546, 2486.773487, 9180.297745],
        13: [4844.082155, 1247.496358, 9977.939894],
        14: [5799.812932, 3939.750578, 8438.994633],
        15: [4774.172495, 2561.964214, 4977.603299],
        16: [4698.885169, 2982.058156, 7392.274638],
        17: [5052.616098, 4900.487158, 2202.164446],
        18: [4697.092614, 6292.276653, 1738.6029],
        19: [3954.675928, 2249.329085, 10126.20052],
        20: [3368.211559, 4350.103211, 6341.601026],
        21: [2275.350296, 2058.764732, 9266.288906],
        22: [6348.036119, 2952.529814, 10122.2469],
        23: [3842.043698, 4089.827617, 7974.444682],
        24: [73194, 15978.7, 35029.3],
        25: [5878.386609, 2172.311862, 5313.66071],
    },
}

scales = {
    "10-01": np.array([298.66187, 301.37174, 1050.67223]),
    "8-01": np.array([298.75923295, 304.41589844, 988.40414663]),
}


type_to_date = {"validation": "10-01", "test": "8-01"}

In [23]:
base_path = Path(
    "/Users/thomasathey/Documents/mimlab/mouselight/benchmarking_datasets"
)


In [24]:
im_path = base_path / "test_1-gfp.tif"
im = io.imread(im_path, plugin="tifffile")
im = np.swapaxes(im,0,2)

swc_base_path = base_path / "Manual-GT"

f = im_path.parts[-1][:-8].split("_")
image = f[0]
date = type_to_date[image]
num = int(f[1])

scale = scales[date]
brain_offset = brain_offsets[date]
vol_offset = vol_offsets[date][num]
im_offset = np.add(brain_offset, vol_offset)

lower = int(np.floor((num - 1) / 5) * 5 + 1)
upper = int(np.floor((num - 1) / 5) * 5 + 5)
dir1 = date + "_" + image + "_" + str(lower) + "-" + str(upper)
dir2 = date + "_" + image + "_" + str(num)
swc_path = swc_base_path / dir1 / dir2

swc_files = list(swc_path.glob("**/*.swc"))

In [25]:
paths_total = []
labels_total = np.zeros(im.shape)
for swc_num, swc in enumerate(swc_files):
    if "cube" in swc.parts[-1]:
        # skip the bounding box swc
        continue

    df, swc_offset, _, _, _ = read_swc(swc)

    offset_diff = np.subtract(swc_offset, im_offset)
    G = df_to_graph(df)

    paths = graph_to_paths(G)

    # for every path in that swc
    for path_num, p in enumerate(paths):
        pvox = (p + offset_diff) / (scale) * 1000
        paths_total.append(pvox)

                    

In [26]:
napari_viewer(im,  shapes=paths_total)

In [27]:
for path_voxel in paths_total:
    for voxel_num, voxel in enumerate(path_voxel):
        if voxel_num == 0:
            continue
        voxel_prev = path_voxel[voxel_num-1,:]
        xs,ys,zs = swc2voxel.Bresenham3D(int(voxel_prev[0]), int(voxel_prev[1]), int(voxel_prev[2]),int(voxel[0]), int(voxel[1]), int(voxel[2]))
        for x,y,z in zip(xs,ys,zs):
            vox = np.array((x,y,z))
            if (vox >= 0).all() and (vox < im.shape).all():
                labels_total[x,y,z] = 1


In [28]:
label_flipped = labels_total*0
label_flipped[labels_total==0] = 1
dists = distance_transform_edt(label_flipped, sampling = scale)

labels_total[dists <= 1000] = 1

In [29]:
napari_viewer(im,  labels=labels_total)